In [ ]:
import os
import torch
import numpy as np
from transformers import TrainingArguments, Trainer, GPT2Tokenizer, GPT2LMHeadModel

from generation import TasksDataset, test_tokenization

os.environ["TOKENIZERS_PARALLELISM"] = "true"
answer_token = 41484

model = GPT2LMHeadModel.from_pretrained('gpt2').to("cuda")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer.eos_token = "<|endoftext|>"
tokenizer.pad_token = tokenizer.eos_token
test_tokenization(tokenizer)

In [ ]:
# use only the first few layers out of 12
model._modules["transformer"]._modules["h"] = model._modules["transformer"]._modules["h"][:3]

In [ ]:
final_answer_loss_contribution = 0.5  # between 0 and 1


class TransformerTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        input_ids = inputs.pop("input_ids")

        # batched generation
        lm_logits = model(input_ids).logits

        labels = input_ids.to(lm_logits.device)
        shift_logits = lm_logits[:, :-1, :].contiguous()
        labels = labels[:, 1:].contiguous()

        # cut out the task part (the part before "answer")
        reasoning_shift_logits = []
        reasoning_labels = []
        final_answer_shift_logits = []
        final_answer_labels = []
        for ex_shift_logits, ex_labels in zip(shift_logits, labels):
            # find the indexes of the "answer" token
            answer_index = torch.where(ex_labels == answer_token)[0]
            answer_index = int(answer_index)
            # cut out the task part
            reasoning_shift_logits.append(ex_shift_logits[answer_index:-1])
            reasoning_labels.append(ex_labels[answer_index:-1])
            # loss for the final answer will be calculated separately
            final_answer_shift_logits.append(ex_shift_logits[-1:])
            final_answer_labels.append(ex_labels[-1:])

        # calculate loss only for the tokens after "answer"
        loss_fct = torch.nn.CrossEntropyLoss()
        reasoning_lm_loss = loss_fct(
            torch.cat(reasoning_shift_logits),
            torch.cat(reasoning_labels),
        )
        loss_fct = torch.nn.CrossEntropyLoss()
        final_answer_lm_loss = loss_fct(
            torch.cat(final_answer_shift_logits),
            torch.cat(final_answer_labels),
        )
        return reasoning_lm_loss * (1 - final_answer_loss_contribution) + final_answer_lm_loss * final_answer_loss_contribution

    def save_model(self, output_dir, _internal_call):
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        torch.save(self.model.state_dict(), f"{output_dir}/gpt2_pytorch_model.bin")
        self.tokenizer.save_pretrained(output_dir)

In [ ]:
def evaluate_example(model, ex):
    target_output = torch.cat([ex["task_ids"], ex["reasoning_ids"]]).to("cuda")
    task_len = len(ex["task_ids"])
    out = model.generate(
        input_ids=ex["task_ids"].reshape(1, -1).to("cuda"),
        attention_mask=ex["attention_mask"][:task_len].reshape(1, -1).to("cuda"),
        max_length=len(target_output),
        temperature=1,
        pad_token_id=tokenizer.pad_token_id,
    )
    if len(out[0]) != len(target_output):
        return False
    return all(out[0] == target_output)


def eval_correctness(model, dataset):
    return np.mean([evaluate_example(model, ex) for ex in dataset])

In [ ]:
task_steps_limit = 1
mask = True

trainer = TransformerTrainer(
    model=model,
    tokenizer=tokenizer,
    args=TrainingArguments(
        disable_tqdm=True,  # This disables the progress bars
        learning_rate=5e-4,
        num_train_epochs=1,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=1,
        dataloader_num_workers=2,
        optim="adamw_torch",
        output_dir="out",
        weight_decay=1e-2,
        # logging_steps=10,
        # save_strategy="epoch"
    ),
)
# disable log printing
trainer.log = lambda logs: None


total_examples = 0
while True:
    _num_examples_per_num_steps = [(ns, 16) for ns in range(1, task_steps_limit + 1)]
    trainer.train_dataset = TasksDataset(tokenizer, mask, _num_examples_per_num_steps)
    total_examples += 16 * task_steps_limit
    trainer.train()

    # for each steps length, check whether model answers correctly
    _num_examples_per_num_steps = [(ns, 1) for ns in range(1, task_steps_limit + 1)]
    eval_dataset = TasksDataset(tokenizer, mask, _num_examples_per_num_steps)
    scores = [evaluate_example(model, ex) for ex in eval_dataset]
    # create accuracy bar
    accuracy_bar = "«"
    for score in scores:
        if score:
            accuracy_bar += "█"
        else:
            accuracy_bar += " "
    accuracy_bar += "»"

    print(f"{total_examples:9}  seq.len.: {task_steps_limit:3}  " + accuracy_bar)
    
    if all(scores):
        # all answers were correct, so increase difficulty level
        task_steps_limit += 1